# Somativa 2 - Renato Curvelo

In [134]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [183]:
# Selecionando as colunas mais importantes
colunas_selecionadas = ['Rented Bike Count', 'Temperature(°C)', 'Hour']
df = pd.read_excel('seoul_bike_data.xlsx')
df_reduzido = df[colunas_selecionadas]
df_reduzido.describe()

,Rented Bike Count,Temperature(°C),Hour
count,8760.000000,8760.000000,8760.000000
mean,704.602055,12.882922,11.500000
std,644.997468,11.944825,6.922582
min,0.000000,-17.800000,0.000000
25%,191.000000,3.500000,5.750000
50%,504.500000,13.700000,11.500000
75%,1065.250000,22.500000,17.250000
max,3556.000000,39.400000,23.000000


In [185]:
# Dividir os dados em treino e teste e treinando o modelo
X_train, X_test, y_train, y_test = train_test_split(df_reduzido.drop(columns=['Rented Bike Count']) , df_reduzido['Rented Bike Count'], test_size=0.2, random_state=42)

# Criar o pipeline
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),  # Etapa de preparação
    ('regressor', xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_estimators=100))  # Modelo
])

# Treinar o pipeline
pipe.fit(X_train, y_train)

# Fazer predições
y_pred = pipe.predict(X_test)

* No pipeline acima, não inclui um encoder, visto que nenhuma variável categórica era relevante o suficiente para entrar no dataframe reduzido.
* Como não haviam outliers nessas colunas, escolhi utilzar o StandardScaler.

In [187]:
# Calcular métricas
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Mostrar métricas
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error: {mse}')

# Comparar valores reais com predições
comparison = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print(comparison.head(15))


Root Mean Squared Error (RMSE): 425.0931547644125
Mean Absolute Error (MAE): 265.1030595237262
Mean Squared Error: 180704.19022756073
    Actual    Predicted
0      963  1603.263794
1      668   652.012451
2      542   455.743774
3     2034  1877.826660
4     1115   537.434082
5      886   742.508118
6      308   349.604980
7      155   227.408035
8      451  1412.332153
9      572   362.337463
10     154   151.133987
11      40    85.337021
12     294   365.116852
13    1395   895.613586
14    1097   963.070374


# Análise dos Resultados Atualizados
_Os valores de erro mostram que o modelo apresenta um desempenho moderado, com algumas limitações:_

## 1. Métricas:
* **Root Mean Squared Error (RMSE): 425.09 =>** O RMSE está alto. Na média, as predições estão numa distância considerável dos valores reais.

* **Mean Absolute Error (MAE): 265.10 =>** O modelo está errando em 265 bicicletas aproximadamente. Para mais ou para menos. Visto que a média é 500 e a maior parte dos dias varia entre 250 e 1000 bicicletas, até que não está tão ruim, mas pode melhorar.

* **Mean Squared Error (MSE): 180704.19 =>** Reflete o impacto de erros grandes, pois amplifica as discrepâncias maiores. Existem vários outliers na coluna de bicicletas alugadas, talvez analizar melhor esses dias/horários para identificar os motivos seria de ajuda.

## 2. Predições:
O modelo captura algumas tendências, como nas linhas 1, 3 e 13, mas ainda tem dificuldades em prever os valores em outros casos, como nas linhas 0 e 8.